In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import keras.backend as K

In [ ]:
def load_models(up,dn):
    model_up = keras.models.load_model(up)
    model_dn = keras.models.load_model(dn)
    return model_up, model_dn

In [ ]:
model_up, model_dn = load_models("multi-task_ecfp_models/MCF7_multi_task_model_ecfp_up.h5","multi-task_ecfp_models/MCF7_multi_task_model_ecfp_dn.h5")

In [ ]:
def load_data():
    gene_list_up = pd.read_csv('MCF7_multi_task_ecfp_gene_list_up.txt',header=None ,delimiter = "\t")
    gene_list_dn = pd.read_csv('MCF7_multi_task_ecfp_gene_list_dn.txt',header=None ,delimiter = "\t")
    genes_up = gene_list_up[0].tolist()
    genes_dn = gene_list_dn[0].tolist()

    df = pd.read_csv("chembl_ecfp.csv")
    ecfp = df['ecfp'].tolist()
    smiles = df['smiles'].tolist()
    
    return genes_up,genes_dn,ecfp, smiles

In [ ]:
genes_up,genes_dn,ecfp, smiles = load_data()

In [ ]:
def load_target():
    
    gene_target_up = pd.read_csv('harmonizome_dn_binarized_use_for_up_model.csv')
    gene_target_up1 = gene_target_dn[gene_target_dn['disease1'] == 'Breast Cancer_Mammary Gland Tissue_GSE1872']
    gene_target_up2 = gene_target_dn[gene_target_dn['disease1'] == 'Breast Cancer_Mammary gland_GSE2528']
    gene_target_up3 = gene_target_dn[gene_target_dn['disease1'] == 'Breast Cancer_Mammary Gland Tissue_GSE2429']
    gene_target_up4 = gene_target_dn[gene_target_dn['disease1'] == 'Breast Cancer_Mammary Gland Tissue_GSE3744']
    gene_target_up5 = gene_target_dn[gene_target_dn['disease1'] == 'Breast Cancer_Mammary Gland Tissue_GSE1378']
    gene_target_up6 = gene_target_dn[gene_target_dn['disease1'] == 'Breast Cancer_Mammary Gland Tissue_GSE1379']
    
    
    gene_target_dn = pd.read_csv('harmonizome_dn_binarized_use_for_up_model.csv')
    gene_target_dn1 = gene_target_dn[gene_target_dn['disease1'] == 'Breast Cancer_Mammary Gland Tissue_GSE1872']
    gene_target_dn2 = gene_target_dn[gene_target_dn['disease1'] == 'Breast Cancer_Mammary gland_GSE2528']
    gene_target_dn3 = gene_target_dn[gene_target_dn['disease1'] == 'Breast Cancer_Mammary Gland Tissue_GSE2429']
    gene_target_dn4 = gene_target_dn[gene_target_dn['disease1'] == 'Breast Cancer_Mammary Gland Tissue_GSE3744']
    gene_target_dn5 = gene_target_dn[gene_target_dn['disease1'] == 'Breast Cancer_Mammary Gland Tissue_GSE1378']
    gene_target_dn6 = gene_target_dn[gene_target_dn['disease1'] == 'Breast Cancer_Mammary Gland Tissue_GSE1379']

    return gene_target_up1,gene_target_up2,gene_target_up3,gene_target_up4,gene_target_up5,gene_target_up6,
gene_target_dn1,gene_target_dn2,gene_target_dn3,gene_target_dn4,gene_target_dn5,gene_target_dn6

In [ ]:
gene_target_up1,gene_target_up2,gene_target_up3,gene_target_up4,gene_target_up5,gene_target_up6,
gene_target_dn1,gene_target_dn2,gene_target_dn3,gene_target_dn4,gene_target_dn5,gene_target_dn6 = load_target()

In [ ]:
def secilen_genes(genes,secilen_genes):
    vector = {}
    vectors= []
    for i in secilen_genes.columns:
        for j in genes:
            if(i == j):
            vector[i] = int(secilen_genes[i])
            
    for q,w in vector.items():
        vectors.append(w)
    vectors=np.array(vectors)

    return vectors

In [ ]:
vector_up1 = secilen_genes(genes_up,gene_target_up1)
vector_up2 = secilen_genes(genes_up,gene_target_up2)
vector_up3 = secilen_genes(genes_up,gene_target_up3)
vector_up4 = secilen_genes(genes_up,gene_target_up4)
vector_up5 = secilen_genes(genes_up,gene_target_up5)
vector_up6 = secilen_genes(genes_up,gene_target_up6)

vector_dn1 = secilen_genes(genes_dn,gene_target_dn1)
vector_dn2 = secilen_genes(genes_dn,gene_target_dn2)
vector_dn3 = secilen_genes(genes_dn,gene_target_dn3)
vector_dn4 = secilen_genes(genes_dn,gene_target_dn4)
vector_dn5 = secilen_genes(genes_dn,gene_target_dn5)
vector_dn6 = secilen_genes(genes_dnz,gene_target_dn6)

In [ ]:
ecfpler = []
smileslar = []
def sanal_tarama(p,s, ecfp, smiles,new_aranan_dn,new_aranan_up):
    
    for k in range(0,len(smiles)):
        bin_ecfp = ecfp[p:s]
        ecfp_new = ecfp[p:s]
        smiles_new = smiles[p:s]
        temp = np.zeros((1000,1024))
        
        for i in range(0,len(bin_ecfp)):
            for j in range(0,1024):
                temp[i,j] = bin_ecfp[i][j]

        pred_list = [x for x in model_up.predict(temp)]
        y_up = np.zeros((pred_list[0].shape[0], len(pred_list)))
        for i in range(len(pred_list)):
            y_up[:, i] = pred_list[i].flatten()
        y_up = np.where(y_up > 0.5, 1, 0)

        pred_list = [x for x in model_dn.predict(temp)]
        y_dn = np.zeros((pred_list[0].shape[0], len(pred_list)))
        for i in range(len(pred_list)):
            y_dn[:, i] = pred_list[i].flatten()
        y_dn = np.where(y_dn > 0.5, 1, 0)

        for kk in range(0,len(y_dn)):
            if(((y_dn[kk]==new_aranan_dn).all()) & ((y_up==new_aranan_up).all()) ):
                
                ecfpler.append(ecfp_new[i])
                smileslar.append(smiles_new[i])
                zinc.append(zinc_id_new[i])
                
            if(((y_dn[kk]==new_aranan_dn).all()) ):
                print(smiles_new[kk])

            if(((y_up[kk]==new_aranan_up).all()) ):
                print(smiles_new[kk])
        p=s
        s=s+1000
        return ecfpler,smileslar

In [ ]:
ecfpler1,smileslar1 = sanal_tarama(0,1000, ecfp, smiles,vector_dn1,vector_up1)
ecfpler2,smileslar2 = sanal_tarama(0,1000, ecfp, smiles,vector_dn2,vector_up2)
ecfpler3,smileslar3 = sanal_tarama(0,1000, ecfp, smiles,vector_dn3,vector_up3)
ecfpler4,smileslar4 = sanal_tarama(0,1000, ecfp, smiles,vector_dn4,vector_up4)
ecfpler5,smileslar5 = sanal_tarama(0,1000, ecfp, smiles,vector_dn5,vector_up5)
ecfpler6,smileslar6 = sanal_tarama(0,1000, ecfp, smiles,vector_dn6,vector_up6)